In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from pyswarms.single import GlobalBestPSO
import time

In [69]:
df = pd.read_csv("Bank_Personal_Loan_Modelling.csv")
df = df = df[['Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Securities Account',
       'CD Account', 'Online', 'CreditCard', 'Personal Loan']]

In [70]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values
n_features = X.shape[1]
n_classes = len(df['Personal Loan'].unique())

In [71]:
sc = StandardScaler()
X = sc.fit_transform(X)

In [72]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=32)


In [86]:
def create_custom_model(input_dim, output_dim, nodes, n=1, name='model'):
    model = Sequential(name=name)
    for i in range(n):
        model.add(Dense(nodes, input_dim=input_dim, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [88]:
n_layers = 2
model = create_custom_model(n_features, n_classes,4, n_layers)
model.summary()

callback = model.fit(X_train, Y_train,batch_size=5,epochs=50,validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 4)                 52        
                                                                 
 dense_28 (Dense)            (None, 4)                 20        
                                                                 
 dense_29 (Dense)            (None, 1)                 5         
                                                                 
Total params: 77
Trainable params: 77
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
700/700 [==============================] - 1s 1ms/step - loss: 0.3604 - accuracy: 0.9051 - val_loss: 0.2584 - val_accuracy: 0.9013
Epoch 2/50
700/700 [==============================] - 1s 952us/step - loss: 0.2222 - accuracy: 0.9051 - val_loss: 0.2182 - val_accuracy: 0.9013
Epoch 3/50
700/700 [=======================

In [89]:
def get_shape(model):
    weights_layer = model.get_weights()
    shapes = []
    for weights in weights_layer:
        shapes.append(weights.shape)
    return shapes
def set_shape(weights,shapes):
    new_weights = []
    index=0
    for shape in shapes:
        if(len(shape)>1):
            n_nodes = np.prod(shape)+index
        else:
            n_nodes=shape[0]+index
        tmp = np.array(weights[index:n_nodes]).reshape(shape)
        new_weights.append(tmp)
        index=n_nodes
    return new_weights

In [90]:
def evaluate_nn(W, shape,X_train=X_train, Y_train=Y_train):
    results = []
    for weights in W:
        model.set_weights(set_shape(weights,shape))
        score = model.evaluate(X_train, Y_train, verbose=0)
        results.append(1-score[1])
    return results

In [91]:
shape = get_shape(model)
x_max = 1.0 * np.ones(83)
x_min = -1.0 * x_max
bounds = (x_min, x_max)
options = {'c1': 0.4, 'c2': 0.8, 'w': 0.4}
optimizer = GlobalBestPSO(n_particles=25, dimensions=83,options=options, bounds=bounds)

In [92]:
cost, pos = optimizer.optimize(evaluate_nn, 15, X_train=X_train, Y_train=Y_train,shape=shape)
model.set_weights(set_shape(pos,shape))
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

2023-04-04 14:40:39,131 - pyswarms.single.global_best - INFO - Optimize for 15 iters with {'c1': 0.4, 'c2': 0.8, 'w': 0.4}
pyswarms.single.global_best: 100%|█████████████████████████████████████████████████████████████|15/15, best_cost=0.0711
2023-04-04 14:41:17,203 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.07114285230636597, best pos: [ 0.07826162  0.08599353  0.36759123  0.33552982  0.4364281   0.10284238
 -0.17047119 -0.28294752 -0.06587384 -0.74869446  0.42570986 -0.36733672
 -0.18804828  0.23537119  0.28025729  0.54059031 -0.19130446  0.21255572
 -0.4416968   0.13830106  0.54101305  0.12174376  0.49533318 -0.20430659
 -0.66517986 -0.27894125  0.1397895   0.48863598  0.18637788  0.53230857
  0.21285894  0.2650344   0.21655457  0.38780765  0.30921243 -0.75871579
 -0.42921225  0.03102575  0.57815825  0.47379541  0.43781004 -0.51722708
 -0.30614556 -0.0317794  -0.05788864  0.31467746  0.48668233  0.33408285
  0.19928988  0.71918345  0.7300867   0.5037

47/47 [==============================] - 0s 674us/step - loss: 0.3373 - accuracy: 0.9180
Test loss: 0.33733120560646057
Test accuracy: 0.9179999828338623
